# Dimension reduction


## Background

Individual remote sensing images can be affected by noisy data, including clouds, cloud shadows, and haze. 
To produce cleaner images that can be compared more easily across time, we can create 'summary' images or 'composites' that combine multiple images into one by reducing dimensions.

Some methods for generating composites include estimating the `median`, `mean`, `minimum`, or `maximum` pixel values in an image.
Care must be taken with these, as they do not necessarily preserve spectral relationships across bands. 
To learn how to generate a composite that does preserve these relationships, see the [Generating geomedian composites notebook](Generating_geomedian_composites.ipynb).

## Description
This notebook demonstrates how to generate a number of different composites from satellite images, and discusses the uses of each.
Specifically, this notebook demonstrates how to generate:

1. Median composites
2. Mean composites
3. Minimum and maximum composites
4. Nearest-time composites
5. Perform a PCA

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

In [ ]:
%matplotlib inline

#import datacube
import matplotlib.pyplot as plt
import xarray as xr
from dea_tools.datahandling import load_ard, first, last, nearest
from dea_tools.bandindices import calculate_indices#
from dea_tools.plotting import rgb
import pystac_client
import planetary_computer
import odc.stac
from pystac.extensions.eo import EOExtension as eo

In [ ]:
#!pip install dea-tools

## Load Landsat 8 data

Here we load a timeseries of cloud-masked Landsat 8 satellite images through the datacube API using the [load_ard](Using_load_ard.ipynb) function.

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

bbox_of_interest = [9.9805, 49.8597, 10.1261, 49.9556]
time_of_interest = "2021-01-01/2021-12-31"

search = catalog.search(
    collections=["landsat-c2-l2"],
    bbox=bbox_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}, # filtering with cloud limit 10 %
          "platform": {"in": ["landsat-8"]}} # here we filter Landsat 8 tiles
)

items = search.item_collection()
print(f"Returned {len(items)} Items")

In [ ]:
data = odc.stac.stac_load(items, 
                          bands=["green","red","blue","nir08","qa_pixel"], 
                          bbox=bbox_of_interest,
                          #chunks={},
                          groupby="solar_day")#.isel(time=0)
data

### Cloud masking
Here we utilize the `qa_pixel` band with the binary coded quality flags. [Humboldt Univerity of Berlin](https://pages.cms.hu-berlin.de/EOL/gcg_eo/02_data_quality.html) gives further information on using the quality flags.

In [ ]:
# used qa values
mask_val = [21762, 21890, 21952, 22018, 22080, 22280, 23826, 23888, 24082, 24144, 55052]
# write maseed dataset `data_clean`
data_clean = data.where(data.qa_pixel.isin(mask_val) == False)

In [ ]:
# For decoding decimal values you can use this function
def dec_to_bin(x):
    return int(bin(x)[2:])

for i in mask_val:
    print(dec_to_bin(i))

## Plot timesteps in true colour

To visualise the data, use the pre-loaded `rgb` utility function to plot a true colour image for a series of timesteps. 
White areas indicate where clouds or other invalid pixels in the image have been masked.

The code below will plot three timesteps of the time series we just loaded:

In [ ]:
# Set the timesteps to visualise
timesteps =  [2, 3, 5]

# Generate RGB plots at each timestep
# For the rgb image a stretch all pixel values are used,i.e.
# if you load cloudy and non cloudy images together, it can
# be useful to use "percentile_stretch" attribute to cut
# extreme values for visualization purposes
rgb(data_clean, 
    index=timesteps, 
    bands=['red', 'green','blue'], 
    #percentile_stretch=(0.1,0.99) 
   )

### Compare with not masked data

In [ ]:
# Set the timesteps to visualise
timesteps =  [2, 3, 5]

rgb(data, 
    index=timesteps, 
    bands=['red', 'green','blue'], 
    #percentile_stretch=(0.1,0.99) 
   )

## Median composites

One of the key reasons for generating a composite is to replace pixels classified as clouds with realistic values from the available data. 
This results in an image that doesn't contain any clouds.
In the case of a median composite, each pixel is selected to have the median (or middle) value out of all possible values.

Care should be taken when using these composites for analysis, since the relationships between spectral bands are not preserved.
These composites are also affected by the timespan they're generated over.
For example, the median pixel in a single season may be different to the median value for the whole year.

> **Note:** For an advanced compositing method that maintains spectral relationships between satellite bands, refer to the [Generating geometric median composites](./Generating_geomedian_composites.ipynb) notebook.

### Generating a single median composite from all data

To generate a single median composite, we use the `xarray.median` method, specifying `'time'` as the dimension to compute the median over.

In [ ]:
# Compute a single median from all data
ds_median = data_clean.median('time') # you can change to "data.median('time')" to build a non cloud masked image

# View the resulting median
rgb(ds_median, 
    bands=['red', 'green', 'blue'],
    #percentile_stretch=(0.0001,0.9999)
   )

Visualize different composites, e.g. standard deviation to show the spectral variabiltiy in different measurements

In [ ]:
# Compute a single standard deviation from all data
ds_std = data_clean.std('time') # you can change to "data.median('time')" to build a non cloud masked image

# View the resulting median
rgb(ds_std, 
    bands=['red', 'green', 'blue'],
    #percentile_stretch=(0.0001,0.9999)
   )

### Generating multiple median composites based on length of time
Rather than using all the data to generate a single median composite, it's possible to use the `xarray.resample` method to group the data into smaller time-spans and generate medians for each of these.
Some resampling options are
* `'nD'` - number of days (e.g. `'7D'` for seven days)
* `'nM'` - number of months (e.g. `'6M'` for six months)
* `'nY'` - number of years (e.g. `'2Y'` for two years)

If the area is particularly cloudy during one of the time-spans, there may still be masked pixels that appear in the median.
This will be true if that pixel is always masked.

In [ ]:
# Generate a median by binning data into six-monthly time-spans
ds_resampled_median = data_clean.resample(time='6M').median('time')

# View the resulting medians
rgb(ds_resampled_median, index=[1, 2], bands=['red', 'green', 'blue'])

### Group By
Similar to resample, grouping works by looking at part of the date, but ignoring other parts. For instance, `'time.month'` would group together all January data together, no matter what year it is from.

Some examples are:
 * `'time.day'` - groups by the day of the month (1-31)
 * `'time.dayofyear'` - groups by the day of the year (1-365)
 * `'time.week'` - groups by week (1-52) 
 * `'time.month'` - groups by the month (1-12)
 * `'time.season'` - groups into 3-month seasons:
     - `'DJF'` December, Jaunary, February
     - `'MAM'` March, April, May
     - `'JJA'` June, July, August
     - `'SON'` September, October, November
 * `'time.year'` - groups by the year

In [ ]:
# Generate a median by binning data into six-monthly time-spans
ds_groupby_season = data_clean.groupby('time.season').median()

# View the resulting medians
rgb(ds_groupby_season, col='season', bands=['red', 'green', 'blue'])

## Mean composites

Mean composites involve taking the average value for each pixel, rather than the middle value as is done for a median composite.
Unlike the median, the mean composite can contain pixel values that were not part of the original dataset.
Care should be taken when interpreting these images, as extreme values (such as unmasked cloud) can strongly affect the mean.

### Generating a single mean composite from all data

To generate a single mean composite, we use the `xarray.mean` method, specifying `'time'` as the dimension to compute the mean over.

> **Note**: If there are no valid values for a given pixel, you may see the warning:
`RuntimeWarning: Mean of empty slice`. The composite will still be generated, but may have blank areas.

In [ ]:
# Compute a single mean from all data
ds_mean = data_clean.mean('time')

# View the resulting mean
rgb(ds_mean, bands=['red', 'green', 'blue'])

### Generating multiple mean composites based on a length of time
As with the median composite, it's possible to use the `xarray.resample` method to group the data into smaller time-spans and generate means for each of these.
See the previous section for some example resampling time-spans.

> **Note:** If you get the warning `RuntimeWarning: Mean of empty slice`, this just means that for one of your groups there was at least one pixel that contained all `nan` values.

In [ ]:
# Generate a mean by binning data into six-monthly time-spans
ds_resampled_mean = data_clean.resample(time='6M').mean('time')

# View the resulting medians
rgb(ds_resampled_mean, index=[1,2], bands=['red', 'green', 'blue'])

## Minimum and maximum composites

These composites can be useful for identifying extreme behaviour in a collection of satellite images.

For example, comparing the maximum and minimum composites for a given band index could help identify areas that take on a wide range of values, which may indicate areas that have high variability over the time-line of the composite.

To demonstrate this, we start by calculating the normalised difference vegetation index (NDVI) for our data, which can then be used to generate the maximum and minimum composites.

In [ ]:
# Start by calculating NDVI
data["ndvi"] = (data.nir08-data.red)/(data.nir08+data.red)
data

### Maximum composite

To generate a single maximum composite, we use the `xarray.max` method, specifying `'time'` as the dimension to compute the maximum over.

In [ ]:
# Compute the maximum composite
# with high cloud laod this will cause runtime warnings
da_max = data.ndvi.max('time')

# View the resulting composite
da_max.plot(vmin=-1, vmax=0.7, cmap='RdYlGn');

### Minimum composite

To generate a single minimum composite, we use the `xarray.min` method, specifying `'time'` as the dimension to compute the minimum over.

In [ ]:
# Compute the minimum composite
da_min = data.ndvi.min('time')

# View the resulting composite
da_min.plot(vmin=-0.5, vmax=0.2, cmap='RdYlGn');

## Nearest-time composites

To get an image at a certain time, often there is missing data, due to clouds and other masking.  We can fill in these gaps by using data from surrounding times.

To generate these images, we can use the custom functions `first`, `last` and `nearest` from the [dea_datahandling](../Scripts/dea_datahandling.py#716) script.

You can also use the in-built `.first()` and `.last()` methods when doing `groupby` and `resample` as described above. They are described in the [xarray documentation](http://xarray.pydata.org/en/stable/groupby.html#first-and-last) on grouped data.

### Most-recent composite

Sometime we may want to determine what the landscape looks like by examining the most recent image.  If we look at the last image for our dataset, we can see there is lots of missing data in the last image.

In [ ]:
# Plot the last image in time
rgb(data_clean, index=-1, bands=['red', 'green', 'blue'])

We can calculate how much of the data is missing in this most recent image

In [ ]:
last_blue_image = data_clean.blue.isel(time=-1)

precent_valid_data = float(last_blue_image.count() / 
                           last_blue_image.size) * 100
print(f'The last image contains {precent_valid_data:.2f}% data.')

In order to fill in the gaps and produce a complete image showing the most-recent satellite acquistion for every pixel, we can run the `last` function on one of the arrays.
This will return the most recent cloud-free value that was observed by the satellite for every pixel in our dataset.

In [ ]:
last_blue = last(data_clean.blue, dim='time') # wont work with dask chunks
last_blue.plot(robust=True);

To see how recent each pixel is, we can compare the age of the pixels with the latest value we have.

In [ ]:
# Identify latest time in our data
last_time = last_blue.time.max()

# Compare the timestamps and convert them to number of days for plotting.
num_days_old = (last_time - last_blue.time).dt.days
num_days_old.plot(cmap='plasma', size=6);

# yes, 70 days 

We can run this method on all of the bands. 
However we only want pixels that have data in every band. 
On the edges of a satellite pass, some bands don't have data.

To get rid of pixels with missing data, we will convert the dataset to an array, and select only those pixels with data in all bands.

In [ ]:
# Convert to array
da = data_clean.to_array(dim='variable')

# Create a mask where data has no-data
no_data_mask = da.isnull().any(dim='variable')

# Mask out regions where there is no-data
da = da.where(~no_data_mask)


Now we can run the `last` function on the array, then turn it back into a dataset. 

In [ ]:
da_latest = last(da, dim='time')

ds_latest = da_latest.to_dataset(dim='variable').drop_dims('variable')

# View the resulting composite
rgb(ds_latest, bands=['red', 'green', 'blue'])

### Before and after composites

Often it is useful to view images before and after an event, to see the change that has occured.

To generate a composite on either side of an event, we can split the dataset along time.

We can then view the composite `last` image before the event, and the composite `first` image after the event.

In [ ]:
# Dates here are inclusive. Use None to not set a start or end of the range.
event_time = '2021-06-01'
before_event = slice(None, event_time)
after_event = slice(event_time, None)

# Select the time period and run the last() or first() function on every band.
da_before = last(da.sel(time=before_event), dim='time')
da_after = first(da.sel(time=after_event), dim='time')

# Convert both DataArrays back to Datasets for plotting
ds_before = da_before.to_dataset(dim='variable').drop_dims('variable')
ds_after = da_after.to_dataset(dim='variable').drop_dims('variable')

The composite image before the event, up to `2021-06-01`:

In [ ]:
rgb(ds_before,['red', 'green', 'blue'])

The composite image after the event, from `2020-06-01` onward:

In [ ]:
rgb(ds_after,['red', 'green', 'blue'])

### Nearest time composite

Sometimes we just want the closest available data to a particular point in time. 
This composite will take values from before _or_ after the specified time to find the nearest observation to our target time:

In [ ]:
# Generate nearest time composite
da_nearest = nearest(da, dim='time', target=event_time)

# Plot nearest time composite
ds_nearest = da_nearest.to_dataset(dim='variable').drop_dims('variable')
rgb(ds_nearest,['red', 'green', 'blue'])

By looking at the `time` for each pixel, we can see if the pixel was taken from before or after the target time.

In [ ]:
target_datetime = da_nearest.time.dtype.type(event_time)

# Calculate different in times and convert to number of days
num_days_different = (da_nearest.time.min(dim='variable') - target_datetime).dt.days

# Plot days before or after target date
num_days_different.plot(cmap='bwr', levels=11, figsize=(6, 6));

## Reducing spectral dimensions with PCA

Principal Component Analysis (PCA) is a popular technique for dimensionality reduction. It can be used to explore patterns in high-dimensional data and assist unsupervised learning.

Principal components are a series of linear combinations of the original variables, among which the first principal component accounts for the greatest variance within a dataset. Each subsequent principal component accounts for the next greatest possible variance and is uncorrelated with the previously defined components.

This technique is useful also very for understanding Sentinel-2 data as images are captured in 12 spectral bands but only 3 variables can be visualized in a RGB composite. PCA can also be applied to timeseries data to investigate temporal evolution patterns for different land cover types.

In [ ]:
%matplotlib inline

import datacube
from sklearn.decomposition import PCA

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.datahandling import load_ard
from dea_tools.plotting import rgb
from dea_tools.classification import sklearn_flatten, sklearn_unflatten
# package installing using
# !pip install <PACKAGE01> <PACKAGE02>

#### Load data and mask clouds

In [ ]:
data = odc.stac.stac_load(items, 
                          bands=["green","red","blue","nir08",
                                 "swir16","swir22","qa_pixel"], 
                          bbox=bbox_of_interest,
                          #chunks={},
                          groupby="solar_day")#.isel(time=0)
data

In [ ]:
data_clean = data.where(data.qa_pixel.isin(mask_val) == False)

In [ ]:
# Visualize data using selected input spectral bands
rgb(data_clean,
    bands=['swir22', 'nir08', 'blue'],
    col='time',
    col_wrap=3)


### Applying PCA to transform and visualize data
To perform a PCA, data is first transformed into a numpy array that can be used by sklearn using the DEA function sklearn_flatten.
We drop `qa_pixel` since it is not directly carrying spectral information.

In [ ]:
data_clean = data_clean.drop(['qa_pixel'])
x = sklearn_flatten(data_clean)

A PCA model is generated with 3 principal components and fitted on the data.

In [ ]:
pca = PCA(n_components=3)
pca.fit(x)

We can investigate how much variance is accounted for in each principal component. In the default example, the first principal component accounts for a much high variance than the next two.

This step can help determine whether more principal components are needed.

In [ ]:
print('Relative variance in principal components:',
      pca.explained_variance_ratio_)


The input data can now be transformed into this new reference space and rearranged into an `xarray.Dataset` compatible with our input data.

In [ ]:
predict = pca.transform(x)
out = sklearn_unflatten(predict, data_clean)
out = out.to_dataset(dim=out.dims[0]).transpose('time', 'y', 'x')

Plot single principal components by renaming the bands

In [ ]:
out['pca1'] = out[0]
out['pca2'] = out[1]
out['pca2'] = out[2]

In [ ]:
out.pca1.isel(time=2).plot()

### Visualize PCA results as rgb

In [ ]:
# Plot PCA bands
rgb(out,
    bands=[1, 0, 2],
    col='time',
    col_wrap=3,
    percentile_stretch=[0.2, 0.90])


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.


**Last modified:** June 2023